In [ ]:
import numpy as np 
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Veri setinin dizinini tanımlayın
direc = Path('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset')
filepaths = list(direc.glob(r'**/*.png'))  # Tüm PNG dosyalarının yollarını al

In [ ]:
# Etiketleri oluşturma
Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

In [ ]:
# DataFrame oluşturma
filepaths = pd.Series(filepaths, name='FilePaths').astype(str)
Labels = pd.Series(Labels, name='Labels').astype(str)

In [ ]:
# img_df oluşturma
img_df = pd.merge(filepaths, Labels, right_index=True, left_index=True)

In [ ]:
# "GT" etiketli dosyaları hariç tutma
img_df = img_df[img_df['Labels'].apply(lambda x: x[-2:] != 'GT')]

In [ ]:
# Veriyi karıştırma
img_df = img_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Sonuçları kontrol etme
print("img_df Head:")
print(img_df.head())

In [ ]:
# Verileri eğitim ve test setlerine ayırma
train_df, test_df = train_test_split(img_df, test_size=0.2, random_state=42)

In [ ]:
# Dosya yollarının string formatında olduğundan emin olalım
train_df['FilePath'] = train_df['FilePaths'].astype(str)
test_df['FilePath'] = test_df['FilePaths'].astype(str)

In [ ]:
# Eğitim ve test veri çerçevelerinin etiketlerini ayarlama
train_df['Label'] = train_df['Labels'].astype(str)
test_df['Label'] = test_df['Labels'].astype(str)

In [ ]:
# Sonuçları kontrol edelim
print("Train DataFrame Head:")
print(train_df.head())
print("\nTest DataFrame Head:")
print(test_df.head())

In [ ]:
# Veri artırma ve ölçeklendirme
train_datagen = ImageDataGenerator(rescale=1./255, 
                                    rotation_range=20, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    shear_range=0.2, 
                                    zoom_range=0.2, 
                                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Eğitim verisi için veri akışı
print("Train DataFrame Columns:", train_df.columns)  # Sütunları kontrol et

In [ ]:
# Eğitim verisi için veri akışı
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='FilePath',  # Dosya yolu sütunu
    y_col='Labels',    # Etiket sütunu
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)


In [ ]:
print("Test DataFrame Columns:", test_df.columns)

In [ ]:
# Test veri artırma için ImageDataGenerator tanımlama
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Test verisi için veri akışı
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='FilePath',  # Burada doğru sütun adını kullandığınızdan emin olun
    y_col='Labels',    # Doğru etiket sütununu kullanmalısınız
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Model tanımlama
model = Sequential()

# İlk katman: Girdi katmanı
model.add(Input(shape=(128, 128, 3)))

# İlk konvolüsyon katmanı
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# İkinci katman
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Üçüncü katman
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Düzleştirme katmanı
model.add(Flatten())

# Tam bağlantılı katmanlar
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))  # 9 sınıf için çıkış katmanı

# Model derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(train_generator, validation_data=test_generator, epochs=10)


##### Konvolüsyon Katmanları: Görüntüdeki özellikleri öğrenmek için kullanılır. Conv2D katmanı, 2D konvolüsyon işlemi uygular.
##### MaxPooling Katmanları: Boyutu azaltarak daha az parametre ile daha iyi özellikler öğrenmek için kullanılır.
##### Flatten Katmanı: Çok boyutlu veriyi düzleştirir; konvolüsyon katmanlarından gelen çıktıyı tamamen bağlı (dense) katmanlara beslemek için gereklidir.
##### Dense Katmanları: Son sınıflandırma işlemi için kullanılır. Son katmanda softmax aktivasyonu, sınıflandırma için kullanılır.

In [ ]:
import matplotlib.pyplot as plt

# Eğitim süreci sonuçlarını görselleştirme
plt.figure(figsize=(12, 6))

# Doğruluk grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Accuracy Fonksiyonu - Eğitim ve Doğrulama')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()

# Kayıp fonksiyonu (Loss) için grafik
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Loss Fonksiyonu - Eğitim ve Doğrulama')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()

plt.tight_layout()
plt.show()
